In [1]:
import sys

sys.path.append("../")
sys.path.append("../../")

from difference_matrix import Difference_Matrix

import numpy as np

from prior_models import normal_prior

### General Performance tests for matrix inversion algorithms

Let's create a second order difference matrix

In [2]:
D=Difference_Matrix(n=100,k=1)

In [3]:
D.D

array([[ 1., -2.,  1., ...,  0.,  0.,  0.],
       [ 0.,  1., -2., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  0., ..., -2.,  1.,  0.],
       [ 0.,  0.,  0., ...,  1., -2.,  1.]])

In [4]:
D.compute_k_difference(0)

array([[-1.,  1.,  0., ...,  0.,  0.,  0.],
       [ 0., -1.,  1., ...,  0.,  0.,  0.],
       [ 0.,  0., -1., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  0., ..., -1.,  1.,  0.],
       [ 0.,  0.,  0., ...,  0., -1.,  1.]])

In [5]:
D.D.shape

(98, 100)

### Confirm DDT inverse computed correctly via LAPACK

In [6]:
assert(np.allclose(D.DDT.dot(D.DDT_inv),np.eye(100-2),rtol=1e-8,atol=1e-8))

In [7]:
D=Difference_Matrix(100,2,style="sparse")

In [8]:
assert(np.allclose(D.DDT.dot(D.DDT_inv),np.eye(100-3),rtol=1e-8,atol=1e-8))

### Large computations of Difference Matrix


In [9]:
D=Difference_Matrix(1000,1,style="lapack")

In [10]:
assert(np.allclose(D.DDT.dot(D.DDT_inv),np.eye(1000-2),rtol=1e-4,atol=1e-4))

In [11]:
D=Difference_Matrix(1000,1,style="sparse")


In [12]:
assert(np.allclose(D.DDT.dot(D.DDT_inv),np.eye(1000-2),rtol=1e-4,atol=1e-4))

### High order difference matrix

In [13]:
D=Difference_Matrix(100,4,style="lapack")
D.D

array([[  1.,  -5.,  10., ...,   0.,   0.,   0.],
       [  0.,   1.,  -5., ...,   0.,   0.,   0.],
       [  0.,   0.,   1., ...,   0.,   0.,   0.],
       ...,
       [  0.,   0.,   0., ...,  -1.,   0.,   0.],
       [  0.,   0.,   0., ...,   5.,  -1.,   0.],
       [  0.,   0.,   0., ..., -10.,   5.,  -1.]])

In [14]:
(D.DDT.dot(D.DDT_inv)-np.eye(100-5)).max()


3.8743019104003906e-05

### Confirm Difference Matrix Recursion works

In [15]:
D=Difference_Matrix(2500,1,style="lapack")
d_21=D.compute_k_difference(21)

In [16]:
D=Difference_Matrix(2500,21,style="sparse")


In [17]:
assert (np.allclose(d_21,D.D))

### Time Difference Matrix Computation

In [18]:
D=Difference_Matrix(100,1,style="lapack")

T=Difference_Matrix(100,k=1,t=np.linspace(0,1,100))

In [19]:
assert(np.allclose(T.DDT.dot(T.DDT_inv),np.eye(100-2),rtol=1e-8,atol=1e-8))

In [20]:
D=Difference_Matrix(100,1,style="sparse")

T=Difference_Matrix(100,k=1,t=np.linspace(0,1,100))

In [21]:
assert(np.allclose(T.DDT.dot(T.DDT_inv),np.eye(100-2),rtol=1e-8,atol=1e-8))

### Edge Cases

In [22]:
D=Difference_Matrix(100,0)

assert(np.allclose(D.DDT.dot(D.DDT_inv),np.eye(100-1),rtol=1e-8,atol=1e-8))

In [23]:
T=Difference_Matrix(100,k=0,t=np.linspace(0,1,100))

In [24]:
D=Difference_Matrix(100,0)
D.D

array([[-1.,  1.,  0., ...,  0.,  0.,  0.],
       [ 0., -1.,  1., ...,  0.,  0.,  0.],
       [ 0.,  0., -1., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  0., ..., -1.,  1.,  0.],
       [ 0.,  0.,  0., ...,  0., -1.,  1.]])

In [25]:
truncated_D=Difference_Matrix(99,0)
truncated_D.D


array([[-1.,  1.,  0., ...,  0.,  0.,  0.],
       [ 0., -1.,  1., ...,  0.,  0.,  0.],
       [ 0.,  0., -1., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  0., ..., -1.,  1.,  0.],
       [ 0.,  0.,  0., ...,  0., -1.,  1.]])

In [26]:
truncated_D.D.dot(D.D)

array([[ 1., -2.,  1., ...,  0.,  0.,  0.],
       [ 0.,  1., -2., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  0., ..., -2.,  1.,  0.],
       [ 0.,  0.,  0., ...,  1., -2.,  1.]])

### Testing the performance of Prior Weighted Matrices

In [31]:
prior=normal_prior.Normal_Prior(100,1,1)
prior.prior

array([ 1.43493512,  2.26022001,  1.38637608,  1.74055899,  1.43576312,
        1.95812307,  1.12933179, -0.12883323,  2.26266074,  0.4715466 ,
        1.42518488,  1.90410006,  0.4990974 ,  1.26400537,  1.99884636,
        1.67867928,  2.0613291 ,  0.51672489,  0.48965717,  1.31532515,
        0.34752943,  0.43695865,  0.10031102,  1.5181793 , -0.13026015,
        1.62150679,  2.53053078,  1.40248519,  0.3498486 ,  1.102059  ,
        0.97457837,  1.00719538,  2.56287812,  2.08088977, -0.61491003,
        1.05164135,  1.7744541 ,  1.51194524, -0.04800725,  3.13538239,
        0.80166565,  1.21548644,  0.73891987, -0.01892348,  0.09071782,
        1.99195706,  2.73318508,  0.16790872,  0.72510214,  1.86445007,
        0.08548066,  1.40354833,  0.23539544,  0.61571   ,  1.26308156,
        2.77073716,  2.07562517,  0.80147137,  1.65313802,  0.40238531,
        2.08142182,  2.66937822,  1.42332425,  0.33974487,  2.05631658,
        0.35047095,  1.96677403,  0.86818926,  0.540454  ,  1.44

In [32]:
P=Difference_Matrix(100,1,prior=prior,style="sparse")
P.D

array([[ 2.26022001, -4.52044003,  2.26022001, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  1.38637608, -2.77275217, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  1.74055899, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.73972225,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ..., -2.7994337 ,
         1.39971685,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.91213066,
        -1.82426131,  0.91213066]])

In [33]:
assert(np.allclose(P.DDT.dot(P.DDT_inv),np.eye(100-2),rtol=1e-8,atol=1e-8))

In [34]:
(P.DDT.dot(P.DDT_inv)-np.eye(100-2)).max()

3.7345163993405706e-10